In [ ]:
import matplotlib
matplotlib.use('Agg')

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)
sess_config = tf.ConfigProto()

import sys
import os
import glob

ADE20K_DATA = '/gpfs01/bethge/data/ADE20K_2016_07_26'
COCO_DATA = '/gpfs01/bethge/share/mscoco/COCO'
PASCAL_VOC_DATA = '/gpfs01/bethge/data/PascalVOC'
MASK_RCNN_MODEL_PATH = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/siamese-mask-rcnn/Mask_RCNN/'
SIAMESE_MASK_RCNN_PATH = '/gpfs01/bethge/home/cmichaelis/projects/2018-03_Siamese_Mask_RCNN/siamese-mask-rcnn/'

if MASK_RCNN_MODEL_PATH not in sys.path:
    sys.path.append(MASK_RCNN_MODEL_PATH)
if SIAMESE_MASK_RCNN_PATH not in sys.path:
    sys.path.append(SIAMESE_MASK_RCNN_PATH)
    
from samples.coco import coco
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize
    
import utils as siamese_utils
import model as siamese_model
import config as siamese_config
import siamese_mrcnn_models as model_zoo
    
import time
import datetime
import random
import numpy as np
import skimage.io
import imgaug
import pickle

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class TrainConfig(siamese_config.Config):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 12
    NUM_CLASSES = 1 + 1
    NAME = 'coco'
    EXPERIMENT = 'example'
    # Reduced image sizes
    TARGET_MAX_DIM = 96
    TARGET_MIN_DIM = 75
    IMAGE_MIN_DIM = 400
    IMAGE_MAX_DIM = 512
    # Reduce model size
    FPN_CLASSIF_FC_LAYERS_SIZE = 512
    FPN_FEATUREMAPS = 256
    # Reduce number of rois at all stages
    RPN_ANCHOR_STRIDE = 1
    RPN_TRAIN_ANCHORS_PER_IMAGE = 64
    POST_NMS_ROIS_TRAINING = 500
    POST_NMS_ROIS_INFERENCE = 500
    TRAIN_ROIS_PER_IMAGE = 100
    DETECTION_MAX_INSTANCES = 30
    MAX_GT_INSTANCES = 30
    # Adapt NMS Threshold
    DETECTION_NMS_THRESHOLD = 0.5
    # Adapt loss weights
    LOSS_WEIGHTS = {'rpn_class_loss': 2.0, 
                    'rpn_bbox_loss': 0.1, 
                    'mrcnn_class_loss': 2.0, 
                    'mrcnn_bbox_loss': 0.5, 
                    'mrcnn_mask_loss': 1.0}
    
config = TrainConfig()
config.display()

In [ ]:
exp_dir = ROOT_DIR # os.path.join(ROOT_DIR, "{}_{}".format(config.NAME.lower(), config.EXPERIMENT.lower()))

In [ ]:
coco_nopascal_classes = [8,10,11,12,13,14,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,60,62,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80]
coco_pascal_classes = np.array(range(1,81))[np.array([i not in coco_nopascal_classes for i in range(1,81)])]

In [ ]:
# train_classes = coco_nopascal_classes
train_classes = np.array(range(1,81))

In [ ]:
# Load COCO/train dataset
coco_train = siamese_utils.IndexedCocoDataset()
coco_train.set_active_classes(train_classes)
coco_train.load_coco(COCO_DATA, "train", year="2017")
coco_train.prepare()
coco_train.build_indices()
coco_train.ACTIVE_CLASSES = train_classes

# Load COCO/val dataset
coco_val = siamese_utils.IndexedCocoDataset()
coco_val.set_active_classes(train_classes)
coco_val.load_coco(COCO_DATA, "val", year="2017")
coco_val.prepare()
coco_val.build_indices()
coco_val.ACTIVE_CLASSES = train_classes

### Simple

In [ ]:
# Create model object in inference mode.
model = siamese_model.SiameseMaskRCNN(mode="training", model_dir=MODEL_DIR, config=config)

In [ ]:
# Load weights trained on Imagenet
try: 
    os.path.exists(os.path.join(model.log_dir, "siamese_mrcnn_0001.h5"))
    list_of_files = glob.glob(os.path.join(model.log_dir,'*.h5')) # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getmtime)
    epoch_index = int(latest_file[-7:-3])
    print('loading', latest_file, '...')

    # set layers trainable for resnet weight loading
    if epoch_index <= 40:
        model.set_trainable("(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")
    elif epoch_index <= 120:
        model.set_trainable("(res4.*)|(bn4.*)|(res5.*)|(bn5.*)|(mrcnn\_.*)|(rpn\_.*)|(fpn\_.*)")          
    else:
        model.set_trainable(".*")
    # load weights            
    model.load_weights(latest_file, by_name=True)
    model.epoch = epoch_index
except:
    print('initializing from imagenet weights ...')
    weights_file = model.get_imagenet_weights(pretraining='imagenet-1k')
    model.load_weights(weights_file, by_name=True)
    model.set_log_dir()




In [ ]:
# file = open(os.path.join(exp_dir, "started.txt"),"w")
# file.close()
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE, epochs=40, layers="heads")
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE, epochs=120, layers="4+")
model.train(coco_train, coco_val, learning_rate=config.LEARNING_RATE/10, epochs=160, layers="all")
file = open(os.path.join(exp_dir, "finished.txt"),"w")
file.close()
print("Done")